In [ ]:
import os
import pandas as pd
import numpy as np
import anndata
import time
import matplotlib.pyplot as plt
import json
import requests
import pickle
import gzip as gz

import sys
sys.path.insert(1, '/home/jw3514/Work/CellType_Psy/src/')
from CellType_PSY import *

In [ ]:
ASD_160_GW = Fil2Dict("../../ASD_Circuits/dat/Unionize_bias/Spark_Meta_160.GeneWeight.csv")
ASD_60_GW = Fil2Dict("../../ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.GeneWeight.csv")
ASD_GW_200 = Fil2Dict("../../UNIMED/dat/Genetics/GeneWeights/Rare/GW.ASD.top200.txt")
ASD_GW_100 = Fil2Dict("../../UNIMED/dat/Genetics/GeneWeights/Rare/GW.ASD.top100.txt")

In [ ]:
GeneExpCorr = pd.read_csv("dat/ISH_MERFISH_Gene_CorssSTR_Corr.v3.csv", index_col=0)
#GeneExpCorr = pd.read_csv("dat/ISH_MERFISH_Gene_CorssSTR_Corr.v3.csv", index_col=0)

In [ ]:
GeneExpCorr.head(2)

In [ ]:
Sibling_Genes = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/Unionize_bias/sibling_weights_LGD_Dmis.csv", 
                           header=None)

In [ ]:
Sibling_Genes_entrez = Sibling_Genes[0].values
Sibling_Genes_entrez = [x for x in Sibling_Genes_entrez if x in GeneExpCorr.index.values]

In [ ]:
ASD_top60_entrez = list(ASD_60_GW.keys())
ASD_top60_entrez = [x for x in ASD_top60_entrez if x in GeneExpCorr.index.values]

In [ ]:
GeneExpCorr.loc[Sibling_Genes_entrez, "V2_V3_CT_Corr"].hist(density=1, label="SIB", alpha=0.5)
GeneExpCorr.loc[ASD_top60_entrez, "V2_V3_CT_Corr"].hist(density=1, label="ASD", alpha=0.5)
plt.legend()

In [ ]:
Sibling_Corr2 = (GeneExpCorr.loc[Sibling_Genes_entrez, "V2_V3_CT_Corr"].values) ** 2
ASD_Corr2 = (GeneExpCorr.loc[ASD_top60_entrez, "V2_V3_CT_Corr"].values) ** 2
plt.hist(Sibling_Corr2, density=1, label="SIB", alpha=0.5)
plt.hist(ASD_Corr2, density=1, label="ASD", alpha=0.5)
plt.legend()

In [ ]:
ASD_60_GW_adj = {}
for k,v in ASD_60_GW.items():
    if k in GeneExpCorr.index.values:
        ASD_60_GW_adj[k] = v * (GeneExpCorr.loc[k, "V2_V3_CT_Corr"]**2)

In [ ]:
#ASD_60_GW_adj.to_csv("dat/Bias/ASD.top60.Adjusted_gw.txt")
Dict2Fil(ASD_60_GW_adj, "dat/Bias/ASD.top60.Adjusted_gw.txt")

In [ ]:
ASD_GW_100_adj = {}
for k,v in ASD_GW_100.items():
    if k in GeneExpCorr.index.values:
        ASD_GW_100_adj[k] = v * (GeneExpCorr.loc[k, "V2_V3_CT_Corr"]**2)
ASD_GW_200_adj = {}
for k,v in ASD_GW_200.items():
    if k in GeneExpCorr.index.values:
        ASD_GW_200_adj[k] = v * (GeneExpCorr.loc[k, "V2_V3_CT_Corr"]**2)

In [ ]:
ASD_160_GW_adj = {}
ASD_160_GW_adj2 = {}
ASD_160_GW_adj3 = {}
for k,v in ASD_160_GW.items():
    if k in GeneExpCorr.index.values:
        ASD_160_GW_adj[k] = v * (GeneExpCorr.loc[k, "V2_V3_CT_Corr"]**2)
        ASD_160_GW_adj2[k] = v * (GeneExpCorr.loc[k, "Two_MERFISH_STR_Corr"]**2)
        ASD_160_GW_adj3[k] = v * (GeneExpCorr.loc[k, "Two_MERFISH_STR_Corr"]**2) * (GeneExpCorr.loc[k, "V2_V3_CT_Corr"]**2)

In [ ]:
Dict2Fil(ASD_160_GW_adj, "dat/Bias/ASD.top160.Adjusted_gw.txt")

## Cell Type

In [ ]:
CellTypesDF = pd.read_csv("dat/CellTypeHierarchy.csv")
Class2Cluster = {}
Subclass2Cluster = {}
for i, row in CellTypesDF.iterrows():
    _cluster, _class, _subclass, _supertype = row
    if _class not in Class2Cluster:
        Class2Cluster[_class] = []
    if _subclass not in Subclass2Cluster:
        Subclass2Cluster[_subclass] = []
    Class2Cluster[_class].append(_cluster)
    Subclass2Cluster[_subclass].append(_cluster)

In [ ]:
ClusterAnn = pd.read_excel("../../data/Allen_Mouse_Brain_Cell_Atlas/SuppTables/41586_2023_6812_MOESM8_ESM.xlsx",
                          sheet_name = "cluster_annotation", index_col="cluster_id_label")
def add_class(BiasDF, ClusterAnn):
    for cluster, row in BiasDF.iterrows():
        BiasDF.loc[cluster, "class_id_label"] = ClusterAnn.loc[cluster, "class_id_label"]
        BiasDF.loc[cluster, "CCF_broad.freq"] = ClusterAnn.loc[cluster, "CCF_broad.freq"]
        BiasDF.loc[cluster, "CCF_acronym.freq"] = ClusterAnn.loc[cluster, "CCF_acronym.freq"]
        BiasDF.loc[cluster, "v3.size"] = ClusterAnn.loc[cluster, "v3.size"]
        BiasDF.loc[cluster, "v2.size"] = ClusterAnn.loc[cluster, "v2.size"]
    return BiasDF

In [ ]:
Cluster_V3_Z2 = pd.read_csv("dat/SC_UMI_Mats//Cluster_Z2Mat_ISHMatch.csv", index_col=0)
max_Z, min_Z = 5, -5
ClusterV3_Z2_Mat_clipped = Cluster_V3_Z2.clip(upper=max_Z, lower=min_Z)

In [ ]:
Cluster_V3_Z2_z1clip3 = pd.read_csv("dat/SC_UMI_Mats/Cluster_Z2Mat_ISHMatch.z1clip3.csv", index_col=0)
max_Z, min_Z = 5, -5
ClusterV3_Z2_Mat_clipped33 = Cluster_V3_Z2_z1clip3.clip(upper=max_Z, lower=min_Z)

In [ ]:
ASD_ClusterV3_Bias_clip3 = ABC_AvgCTZ_Weighted(Cluster_V3_Z2_z1clip3, ASD_60_GW_adj)
ASD_ClusterV3_Bias_clip3 = add_class(ASD_ClusterV3_Bias_clip3, ClusterAnn)
ASD_ClusterV3_Bias_clip3.to_csv("dat/Bias/ASD.ClusterV3.top60.UMI.Z2.z1clip3.csv")

In [ ]:
with open('dat/ClusterV3_Z2_Mat_Genes.pkl', 'wb') as file:
    pickle.dump(ClusterV3_Z2_Mat_clipped.index.values, file)

In [ ]:
ASD_ClusterV3_Bias = ABC_AvgCTZ_Weighted(ClusterV3_Z2_Mat_clipped, ASD_60_GW)
ASD_ClusterV3_Bias = add_class(ASD_ClusterV3_Bias, ClusterAnn)
ASD_ClusterV3_Bias.to_csv("dat/Bias/ASD.ClusterV3.top60.UMI.Z2.csv")

In [ ]:
ASD_ClusterV3_BiasDN = ABC_AvgCTZ_Weighted(ClusterV3_Z2_Mat_clipped, ASD_60_GW_adj)
ASD_ClusterV3_BiasDN = add_class(ASD_ClusterV3_BiasDN, ClusterAnn)
ASD_ClusterV3_BiasDN.to_csv("dat/Bias/ASD.ClusterV3.top60.DN.UMI.Z2.csv")

In [ ]:
ASD_ClusterV3_Bias160DN = ABC_AvgCTZ_Weighted(ClusterV3_Z2_Mat_clipped, ASD_160_GW_adj)
ASD_ClusterV3_Bias160DN = add_class(ASD_ClusterV3_Bias160DN, ClusterAnn)
ASD_ClusterV3_Bias160DN.to_csv("dat/Bias/ASD.ClusterV3.top160.DN.UMI.Z2.csv")

In [ ]:
Class = sorted(Class2Cluster.keys())
DF = ASD_ClusterV3_Bias160DN

sns.set(style="whitegrid", context="talk")

# Calculate medians and sort data by medians
ASD_dat_CB = []
medians = []
for _CT in Class:
    subdf = DF[DF["class_id_label"]==_CT]
    data = [x for x in subdf["EFFECT"].values if x==x]
    ASD_dat_CB.append(data)
    medians.append(np.median(data))

# Sort based on medians
sorted_indices = np.argsort(medians)
sorted_ASD_dat_CB = [ASD_dat_CB[i] for i in sorted_indices]
sorted_Class = [Class[i] for i in sorted_indices]

# Plotting
fig, ax = plt.subplots(dpi=480, figsize=(8, 8))

# Create the boxplot
boxplot = ax.boxplot(sorted_ASD_dat_CB, labels=sorted_Class, vert=False, patch_artist=True)

# Color the boxplot for better aesthetics
colors = sns.color_palette("muted", len(sorted_Class))
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)

# Customize the plot
plt.grid(True, linestyle='--', alpha=0.7)
plt.xlabel('ASD Bias', fontsize=14)
plt.ylabel('Classes', fontsize=14)
plt.title('ASD Bias Across Different Classes', fontsize=16, weight='bold')

# Enhance the grid
ax.grid(True, linestyle='--', alpha=0.7)

# Customize tick labels
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Tight layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
Class = sorted(Class2Cluster.keys())
DF = ASD_ClusterV3_Bias

sns.set(style="whitegrid", context="talk")

# Calculate medians and sort data by medians
ASD_dat_CB = []
medians = []
for _CT in Class:
    subdf = DF[DF["class_id_label"]==_CT]
    data = [x for x in subdf["EFFECT"].values if x==x]
    ASD_dat_CB.append(data)
    medians.append(np.median(data))

# Sort based on medians
sorted_indices = np.argsort(medians)
sorted_ASD_dat_CB = [ASD_dat_CB[i] for i in sorted_indices]
sorted_Class = [Class[i] for i in sorted_indices]

# Plotting
fig, ax = plt.subplots(dpi=480, figsize=(8, 8))

# Create the boxplot
boxplot = ax.boxplot(sorted_ASD_dat_CB, labels=sorted_Class, vert=False, patch_artist=True)

# Color the boxplot for better aesthetics
colors = sns.color_palette("muted", len(sorted_Class))
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)

# Customize the plot
plt.grid(True, linestyle='--', alpha=0.7)
plt.xlabel('ASD Bias', fontsize=14)
plt.ylabel('Classes', fontsize=14)
plt.title('ASD Bias Across Different Classes', fontsize=16, weight='bold')

# Enhance the grid
ax.grid(True, linestyle='--', alpha=0.7)

# Customize tick labels
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Tight layout
plt.tight_layout()

# Show the plot
plt.show()

### Old

In [ ]:
CPM_V3 = pd.read_csv("dat/ExpressionMats/cluster_V3_ExpMat.csv", index_col=0)

In [ ]:
Cluster_V3_CPM_Z1 = pd.read_csv("dat/ExpressionMats/Cluster_V3_Z1Mat.csv", index_col=0)
max_Z, min_Z = 5, -5
Cluster_V3_CPM_Z1 = Cluster_V3_CPM_Z1.clip(upper=max_Z, lower=min_Z)

In [ ]:
Cluster_V3_Z2 = pd.read_csv("dat/Z2Mat.ABCMatch.cluster.10xV3.csv", index_col=0)
max_Z, min_Z = 5, -5
ClusterV3_Z2_Mat_clipped = Cluster_V3_Z2.clip(upper=max_Z, lower=min_Z)

In [ ]:
ASD_ClusterV3_Bias = ABC_AvgCTZ_Weighted(CPM_V3, ASD_160_GW_adj)
ASD_ClusterV3_Bias = add_class(ASD_ClusterV3_Bias, ClusterAnn)
ASD_ClusterV3_Bias.to_csv("dat/Bias/ASD.ClusterV3.top160.DN.CPM.csv")

In [ ]:
ASD_ClusterV3_Bias = ABC_AvgCTZ_Weighted(CPM_V3, ASD_60_GW_adj)
ASD_ClusterV3_Bias = add_class(ASD_ClusterV3_Bias, ClusterAnn)
ASD_ClusterV3_Bias.to_csv("dat/Bias/ASD.ClusterV3.top60.DN.CPM.csv")

In [ ]:
#ASD_ClusterV3_Bias.head(50)

In [ ]:
#ASD_ClusterV3_Bias

In [ ]:
ASD_ClusterV3_Bias = pd.read_csv("dat/Bias/ASD.ClusterV3.top60.CPM.pvalues.csv", index_col=0)

In [ ]:
Class = sorted(Class2Cluster.keys())
DF = ASD_ClusterV3_Bias

sns.set(style="whitegrid", context="talk")

# Calculate medians and sort data by medians
ASD_dat_CB = []
medians = []
for _CT in Class:
    subdf = DF[DF["class_id_label"]==_CT]
    data = [x for x in subdf["EFFECT2"].values if x==x]
    ASD_dat_CB.append(data)
    medians.append(np.median(data))

# Sort based on medians
sorted_indices = np.argsort(medians)
sorted_ASD_dat_CB = [ASD_dat_CB[i] for i in sorted_indices]
sorted_Class = [Class[i] for i in sorted_indices]

# Plotting
fig, ax = plt.subplots(dpi=480, figsize=(8, 8))

# Create the boxplot
boxplot = ax.boxplot(sorted_ASD_dat_CB, labels=sorted_Class, vert=False, patch_artist=True)

# Color the boxplot for better aesthetics
colors = sns.color_palette("muted", len(sorted_Class))
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)

# Customize the plot
plt.grid(True, linestyle='--', alpha=0.7)
plt.xlabel('ASD Bias', fontsize=14)
plt.ylabel('Classes', fontsize=14)
plt.title('ASD Bias Across Different Classes', fontsize=16, weight='bold')

# Enhance the grid
ax.grid(True, linestyle='--', alpha=0.7)

# Customize tick labels
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Tight layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
ASD_ClusterV3_Bias = ABC_AvgCTZ_Weighted(Cluster_V3_CPM_Z1, ASD_160_GW_adj)
ASD_ClusterV3_Bias = add_class(ASD_ClusterV3_Bias, ClusterAnn)
ASD_ClusterV3_Bias.to_csv("dat/Bias/ASD.ClusterV3.top160.DN.CPM.Z1.csv")

In [ ]:
Class = sorted(Class2Cluster.keys())
DF = ASD_ClusterV3_Bias

sns.set(style="whitegrid", context="talk")

# Calculate medians and sort data by medians
ASD_dat_CB = []
medians = []
for _CT in Class:
    subdf = DF[DF["class_id_label"]==_CT]
    data = [x for x in subdf["EFFECT"].values if x==x]
    ASD_dat_CB.append(data)
    medians.append(np.median(data))

# Sort based on medians
sorted_indices = np.argsort(medians)
sorted_ASD_dat_CB = [ASD_dat_CB[i] for i in sorted_indices]
sorted_Class = [Class[i] for i in sorted_indices]

# Plotting
fig, ax = plt.subplots(dpi=480, figsize=(8, 8))

# Create the boxplot
boxplot = ax.boxplot(sorted_ASD_dat_CB, labels=sorted_Class, vert=False, patch_artist=True)

# Color the boxplot for better aesthetics
colors = sns.color_palette("muted", len(sorted_Class))
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)

# Customize the plot
plt.grid(True, linestyle='--', alpha=0.7)
plt.xlabel('ASD Bias', fontsize=14)
plt.ylabel('Classes', fontsize=14)
plt.title('ASD Bias Across Different Classes', fontsize=16, weight='bold')

# Enhance the grid
ax.grid(True, linestyle='--', alpha=0.7)

# Customize tick labels
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Tight layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
ASD_ClusterV3_Bias = ABC_AvgCTZ_Weighted(ClusterV3_Z2_Mat_clipped, ASD_160_GW_adj)
ASD_ClusterV3_Bias = add_class(ASD_ClusterV3_Bias, ClusterAnn)
ASD_ClusterV3_Bias.to_csv("dat/Bias/ASD.ClusterV3.top160.DN.Z2.csv")

In [ ]:
ASD_ClusterV3_Bias_100 = ABC_AvgCTZ_Weighted(ClusterV3_Z2_Mat_clipped, ASD_GW_100_adj)
ASD_ClusterV3_Bias_100 = add_class(ASD_ClusterV3_Bias_100, ClusterAnn)
ASD_ClusterV3_Bias_100.to_csv("dat/Bias/ASD.ClusterV3.top100.DN.Z2.csv")

In [ ]:
Class = sorted(Class2Cluster.keys())
DF = ASD_ClusterV3_Bias_100

sns.set(style="whitegrid", context="talk")

# Calculate medians and sort data by medians
ASD_dat_CB = []
medians = []
for _CT in Class:
    subdf = DF[DF["class_id_label"]==_CT]
    data = [x for x in subdf["EFFECT"].values if x==x]
    ASD_dat_CB.append(data)
    medians.append(np.median(data))

# Sort based on medians
sorted_indices = np.argsort(medians)
sorted_ASD_dat_CB = [ASD_dat_CB[i] for i in sorted_indices]
sorted_Class = [Class[i] for i in sorted_indices]

# Plotting
fig, ax = plt.subplots(dpi=480, figsize=(8, 8))

# Create the boxplot
boxplot = ax.boxplot(sorted_ASD_dat_CB, labels=sorted_Class, vert=False, patch_artist=True)

# Color the boxplot for better aesthetics
colors = sns.color_palette("muted", len(sorted_Class))
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)

# Customize the plot
plt.grid(True, linestyle='--', alpha=0.7)
plt.xlabel('ASD Bias', fontsize=14)
plt.ylabel('Classes', fontsize=14)
plt.title('ASD Bias Across Different Classes', fontsize=16, weight='bold')

# Enhance the grid
ax.grid(True, linestyle='--', alpha=0.7)

# Customize tick labels
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Tight layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
ASD_ClusterV3_Bias2 = ABC_AvgCTZ_Weighted(ClusterV3_Z1_Mat_clipped, ASD_160_GW_adj2)
ASD_ClusterV3_Bias2 = add_class(ASD_ClusterV3_Bias2, ClusterAnn)

In [ ]:
CellTypesDF = pd.read_csv("dat/CellTypeHierarchy.csv")
Class2Cluster = {}
Subclass2Cluster = {}
for i, row in CellTypesDF.iterrows():
    _cluster, _class, _subclass, _supertype = row
    if _class not in Class2Cluster:
        Class2Cluster[_class] = []
    if _subclass not in Subclass2Cluster:
        Subclass2Cluster[_subclass] = []
    Class2Cluster[_class].append(_cluster)
    Subclass2Cluster[_subclass].append(_cluster)

In [ ]:
Class = sorted(Class2Cluster.keys())
DF = ASD_ClusterV3_Bias

sns.set(style="whitegrid", context="talk")

# Calculate medians and sort data by medians
ASD_dat_CB = []
medians = []
for _CT in Class:
    subdf = DF[DF["class_id_label"]==_CT]
    data = [x for x in subdf["EFFECT"].values if x==x]
    ASD_dat_CB.append(data)
    medians.append(np.median(data))

# Sort based on medians
sorted_indices = np.argsort(medians)
sorted_ASD_dat_CB = [ASD_dat_CB[i] for i in sorted_indices]
sorted_Class = [Class[i] for i in sorted_indices]

# Plotting
fig, ax = plt.subplots(dpi=480, figsize=(8, 8))

# Create the boxplot
boxplot = ax.boxplot(sorted_ASD_dat_CB, labels=sorted_Class, vert=False, patch_artist=True)

# Color the boxplot for better aesthetics
colors = sns.color_palette("muted", len(sorted_Class))
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)

# Customize the plot
plt.grid(True, linestyle='--', alpha=0.7)
plt.xlabel('ASD Bias', fontsize=14)
plt.ylabel('Classes', fontsize=14)
plt.title('ASD Bias Across Different Classes', fontsize=16, weight='bold')

# Enhance the grid
ax.grid(True, linestyle='--', alpha=0.7)

# Customize tick labels
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Tight layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
Class = sorted(Class2Cluster.keys())
DF = ASD_ClusterV3_Bias2

sns.set(style="whitegrid", context="talk")

# Calculate medians and sort data by medians
ASD_dat_CB = []
medians = []
for _CT in Class:
    subdf = DF[DF["class_id_label"]==_CT]
    data = [x for x in subdf["EFFECT"].values if x==x]
    ASD_dat_CB.append(data)
    medians.append(np.median(data))

# Sort based on medians
sorted_indices = np.argsort(medians)
sorted_ASD_dat_CB = [ASD_dat_CB[i] for i in sorted_indices]
sorted_Class = [Class[i] for i in sorted_indices]

# Plotting
fig, ax = plt.subplots(dpi=480, figsize=(8, 8))

# Create the boxplot
boxplot = ax.boxplot(sorted_ASD_dat_CB, labels=sorted_Class, vert=False, patch_artist=True)

# Color the boxplot for better aesthetics
colors = sns.color_palette("muted", len(sorted_Class))
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)

# Customize the plot
plt.grid(True, linestyle='--', alpha=0.7)
plt.xlabel('ASD Bias', fontsize=14)
plt.ylabel('Classes', fontsize=14)
plt.title('ASD Bias Across Different Classes', fontsize=16, weight='bold')

# Enhance the grid
ax.grid(True, linestyle='--', alpha=0.7)

# Customize tick labels
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Tight layout
plt.tight_layout()

# Show the plot
plt.show()

## Structures

In [ ]:
def TestBiasCompare(Bias1, Bias2, name1="1", name2="2"):
    REGIONS = list(set(Bias1["REGION"].values))
    REG_COR_Dic = dict(zip(REGIONS, ["#B3B6B7", "#D5DBDB", "#AED6F1", "#F5B041", "#34495E", "#FAD7A0", 
                           "#884EA0", "#E6B0AA", "#33a02c",  "#E74C3C", "#2ECC71", "#D2B4DE", "#85C1E9", "#3498DB"]))
    REGIONS_seq = ['Isocortex','Olfactory_areas', 'Cortical_subplate', 'Hippocampus','Amygdala','Striatum', "Thalamus",
              "Hypothalamus", "Midbrain", "Medulla", "Pallidum", "Pons", "Cerebellum"]
    plt.figure(dpi=300, figsize=(6, 4))

    # Use Seaborn for style and color palette
    sns.set(style="whitegrid", context="talk")
    palette = sns.color_palette("deep")

    JoinDF = Bias1.join(Bias2, how="inner", lsuffix="_{}".format(name1), rsuffix="_{}".format(name2))
    R,P = pearsonr(JoinDF["EFFECT_{}".format(name1)].values, JoinDF["EFFECT_{}".format(name2)].values)
    RR,PP = spearmanr(JoinDF["EFFECT_{}".format(name1)].values, JoinDF["EFFECT_{}".format(name2)].values)
    # Scatter plot with improved aesthetics
    for reg in REGIONS_seq:
        RegDF = JoinDF[JoinDF["REGION_{}".format(name1)]==reg]
        X = RegDF["EFFECT_{}".format(name1)].values
        Y = RegDF["EFFECT_{}".format(name2)].values
        plt.scatter(X, Y, facecolor=REG_COR_Dic[reg], alpha=0.7, s=20, label=reg)
    minX, maxY = min(JoinDF["EFFECT_{}".format(name1)].values), max(JoinDF["EFFECT_{}".format(name2)].values)
    
    Top50_X = JoinDF.sort_values("EFFECT_{}".format(name1), ascending=False).head(50).index.values
    top50_Y = JoinDF.sort_values("EFFECT_{}".format(name2), ascending=False).head(50).index.values 
    STR_Com = len(set(Top50_X).intersection(set(top50_Y)))
    plt.text(x=minX, y=maxY*0.7, s="r=%.2f P<%.1e\nrho=%.2f P<%.1e\nSTR_Comm=%d"%(R, P, RR, PP, STR_Com), 
             fontsize=6, weight='bold')
    # Adding labels, title, and legend
    plt.xlabel(name1, fontsize=14, weight='bold')
    plt.ylabel(name2, fontsize=14, weight='bold')
    #plt.title("Scatter Plot of EFFECT vs. SC_Bias", fontsize=16, weight='bold')
    #plt.legend(fontsize=6)
    plt.legend(fontsize=8, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

    
    # Adding grid lines
    plt.grid(True, linestyle='--', alpha=0.5)

    # Adjusting ticks
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # Display the plot
    plt.tight_layout()
    plt.show()

#### Aggreate ct bias to STR

In [ ]:
MERFISH_STRAnn = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/MERFISH.ISH_Annot.csv", index_col=0)

In [ ]:
def GetBiasesForSTR(STR, MERFISH_STRAnn, ASD_CT_Bias_Cluster, CT_ID_SET, Non_Neuron_Class):
    STR_MERFISH_DF = MERFISH_STRAnn[MERFISH_STRAnn["ISH_STR"]==STR]
    total_bias = 0
    total_bias_neuro = 0
    N_cell = 0
    N_neuro = 0
    for i, row in STR_MERFISH_DF.iterrows():
        cluster_id = row["cluster"]
        if cluster_id not in CT_ID_SET:
            continue
        class_id = row["class"]
        asd_cluster_bias = ASD_CT_Bias_Cluster.loc[cluster_id, "EFFECT"]
        if asd_cluster_bias == asd_cluster_bias:
            total_bias += asd_cluster_bias
            N_cell += 1
            if class_id not in Non_Neuron_Class:
                total_bias_neuro += asd_cluster_bias
                N_neuro += 1
    bias_per_cell = total_bias/N_cell
    bias_per_neuro = total_bias_neuro/N_neuro
    Vol = CCF_V3_STR.loc[STR, "total_voxel_counts (10 um)"]
    bias_per_Vol = total_bias/Vol * 100
    return bias_per_cell, bias_per_neuro, bias_per_Vol
STRs = list(set(MERFISH_STRAnn["ISH_STR"]))
STRs.remove("Not in Connectome")
CT_ID_SET = set(ASD_ClusterV3_Bias.index.values)
Non_Neuron_Class = ["30 Astro-Epen","31 OPC-Oligo","32 OEC","33 Vascular","34 Immune"]
CCF_V3_STR = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/CCF_V3_ISH_MERFISH.csv",
                        index_col="CleanName")

In [ ]:
ASD_ClusterV3_Bias = pd.read_csv("dat/Bias/ASD.ClusterV3.top60.DN.UMI.Z2.csv", index_col=0)

In [ ]:
bias_dat_percell = []
bias_dat_perneuro = []
bias_dat_pervol = []
for STR in STRs:
    bias_percell, bias_per_neuro, bias_per_vol = GetBiasesForSTR(STR, MERFISH_STRAnn, ASD_ClusterV3_Bias,
                                                                 CT_ID_SET, Non_Neuron_Class)
    bias_dat_percell.append(bias_percell)
    bias_dat_perneuro.append(bias_per_neuro)
    bias_dat_pervol.append(bias_per_vol)
    print(STR, bias_percell, bias_per_neuro, bias_per_vol)

In [ ]:
AggV2_ASD_DF = pd.DataFrame(data={"STR2":STRs, "EFFECT_perCell":bias_dat_percell, "EFFECT_perNeuro":bias_dat_perneuro,
                                 "EFFECT_perVol":bias_dat_pervol, })
AggV2_ASD_DF = AggV2_ASD_DF.set_index("STR2")

In [ ]:
AggV2_ASD_DF = AggV2_ASD_DF.sort_values("EFFECT_perNeuro", ascending=False)

In [ ]:
ASD_ISH = pd.read_csv("dat/Bias/ASD.ISH.bias.csv", index_col=0)
ASD_MERFISH_adj = pd.read_csv("dat/Bias/STR/ASD.MERFISH_Allen.CM.ISHMatch.Z2.csv", index_col=0)

In [ ]:
for i, row in AggV2_ASD_DF.iterrows():
    
    STR = "_".join(i.split())
    if STR not in ASD_MERFISH_adj.index.values:
        AggV2_ASD_DF.loc[i, "STR"] = ""
        AggV2_ASD_DF.loc[i, "REGION"] = ""
    else:
        REG = ASD_MERFISH_adj.loc[STR, "REGION"]
        AggV2_ASD_DF.loc[i, "STR"] = STR
        AggV2_ASD_DF.loc[i, "REGION"] = REG
    
    
    AggV2_ASD_DF.loc[i, "EFFECT_ISH"] = ASD_ISH.loc[STR, "EFFECT"]

In [ ]:
AggV2_ASD_DF = AggV2_ASD_DF.sort_values("EFFECT_perNeuro", ascending=False)
AggV2_ASD_DF.head(5)

In [ ]:


R1, P1 = pearsonr(ISH_REG_Bias, MF_REG_Bias)
R2, P2 = spearmanr(ISH_REG_Bias, MF_REG_Bias)
print(spearmanr(ISH_REG_Bias, MF_REG_Bias))
plt.figure(dpi=200, figsize=(6,6))
for Reg in REGIONS_seq:
    ISH_Reg = ASD_ISH[ASD_ISH["REGION"]==Reg]["EFFECT"]
    MF_Reg = AggV2_ASD_DF[AggV2_ASD_DF["REGION"]==Reg]["EFFECT_perCell"]
    X = ISH_Reg.mean()
    Y = MF_Reg.mean()
    plt.scatter(ISH_Reg.mean(), MF_Reg.mean(), color=REG_COR_Dic[Reg])
    plt.text(X, Y, s=Reg, fontsize=15)
XXX = ASD_ISH["EFFECT"].values
YYY = AggV2_ASD_DF["EFFECT_perCell"].values
plt.text(-0.4, -0.2, s="R = %.2f\nrho = %.2f"%(R1, R2), fontsize=15)
plt.xlabel("ISH")
plt.ylabel("MERFISH Per Cell")

In [ ]:
ISH_REG_Bias = []
MF_REG_Bias = []
for Reg in REGIONS_seq:
    ISH_Reg = ASD_ISH[ASD_ISH["REGION"]==Reg]["EFFECT"]
    MF_Reg = AggV2_ASD_DF[AggV2_ASD_DF["REGION"]==Reg]["EFFECT_perNeuro"]
    #print(Reg, ISH_Reg.max() - ISH_Reg.min(), MF_Reg.max() - MF_Reg.min())
    ISH_REG_Bias.append(ISH_Reg.mean())
    MF_REG_Bias.append(MF_Reg.mean())

R1, P1 = pearsonr(ISH_REG_Bias, MF_REG_Bias)
R2, P2 = spearmanr(ISH_REG_Bias, MF_REG_Bias)
print(spearmanr(ISH_REG_Bias, MF_REG_Bias))
plt.figure(dpi=200, figsize=(6,6))
for Reg in REGIONS_seq:
    ISH_Reg = ASD_ISH[ASD_ISH["REGION"]==Reg]["EFFECT"]
    MF_Reg = AggV2_ASD_DF[AggV2_ASD_DF["REGION"]==Reg]["EFFECT_perNeuro"]
    X = ISH_Reg.mean()
    Y = MF_Reg.mean()
    plt.scatter(ISH_Reg.mean(), MF_Reg.mean(), color=REG_COR_Dic[Reg])
    plt.text(X, Y, s=Reg, fontsize=15)
plt.plot([-0.6, 0.5], [-0.6, 0.5], color="grey", alpha=0.3)
XXX = ASD_ISH["EFFECT"].values
YYY = AggV2_ASD_DF["EFFECT_perNeuro"].values
plt.text(-0.4, 0.2, s="R = %.2f\nrho = %.2f"%(R1, R2), fontsize=15)
plt.xlim(-0.6, 0.5)
plt.ylim(-0.6, 0.5)
plt.xlabel("ISH")
plt.ylabel("MERFISH Per Neuro")

In [ ]:
R1, P1 = pearsonr(ISH_REG_Bias, MF_REG_Bias)
R2, P2 = spearmanr(ISH_REG_Bias, MF_REG_Bias)
print(spearmanr(ISH_REG_Bias, MF_REG_Bias))
plt.figure(dpi=200, figsize=(6,6))
for Reg in REGIONS_seq:
    ISH_Reg = ASD_ISH[ASD_ISH["REGION"]==Reg]["EFFECT"]
    MF_Reg = AggV2_ASD_DF[AggV2_ASD_DF["REGION"]==Reg]["EFFECT_perVol"]
    X = ISH_Reg.mean()
    Y = MF_Reg.mean()
    plt.scatter(ISH_Reg.mean(), MF_Reg.mean(), color=REG_COR_Dic[Reg])
    plt.text(X, Y, s=Reg, fontsize=15)
XXX = ASD_ISH["EFFECT"].values
YYY = AggV2_ASD_DF["EFFECT_perVol"].values
plt.text(-0.4, -0.2, s="R = %.2f\nrho = %.2f"%(R1, R2), fontsize=15)
plt.xlabel("ISH")
plt.ylabel("MERFISH Per Vol")

In [ ]:
AggV2_ASD_DF[AggV2_ASD_DF["REGION"]=="Thalamus"]

In [ ]:
pearsonr(AggV2_ASD_DF["EFFECT_perCell"].values, AggV2_ASD_DF["EFFECT_ISH"].values)

In [ ]:
pearsonr(AggV2_ASD_DF["EFFECT_perNeuro"].values, AggV2_ASD_DF["EFFECT_ISH"].values)

In [ ]:
pearsonr(AggV2_ASD_DF["EFFECT_perVol"].values, AggV2_ASD_DF["EFFECT_ISH"].values)

In [ ]:
Bias1 = ASD_ISH
Bias2 = AggV2_ASD_DF.sort_values("EFFECT_perNeuro", ascending=False).set_index("STR")
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
Bias1 = ASD_ISH
Bias2 = AggV2_ASD_DF.sort_values("EFFECT_perVol", ascending=False).set_index("STR")
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

In [ ]:
Bias1 = ASD_ISH
Bias2 = AggV2_ASD_DF.sort_values("EFFECT_perCell", ascending=False).set_index("STR")
STR_Com = len(set(Bias1.head(50).index.values).intersection(set(Bias2.head(50).index.values)))
STR_Com

#### STR Stats

In [ ]:
def GetSTR_stats(STR, MERFISH_STRAnn, CT_ID_SET, Non_Neuron_Class):
    STR_MERFISH_DF = MERFISH_STRAnn[MERFISH_STRAnn["ISH_STR"]==STR]
    N_cell = 0
    N_neuro = 0
    N_glia = 0
    for i, row in STR_MERFISH_DF.iterrows():
        cluster_id = row["cluster"]
        if cluster_id not in CT_ID_SET:
            continue
        class_id = row["class"]

        N_cell += 1
        if class_id not in Non_Neuron_Class:
            N_neuro += 1
        else:
            N_glia += 1
    Vol = CCF_V3_STR.loc[STR, "total_voxel_counts (10 um)"]
    
    CellDen = N_cell/Vol
    NeuroDen = N_neuro/Vol
    GliaDen = N_glia/Vol
    NeuroFrac = N_neuro/N_cell
    Neuro2Glia = N_cell/N_glia
    
    return N_cell, N_neuro, N_glia, CellDen, NeuroDen, GliaDen, NeuroFrac, Neuro2Glia

In [ ]:
dat = []
for STR in STRs:
    N_cell, N_neuro, N_glia, CellDen, NeuroDen, GliaDen, NeuroFrac, Neuro2Glia = GetSTR_stats(STR, MERFISH_STRAnn,
                                                                 CT_ID_SET, Non_Neuron_Class)
    dat.append([STR, N_cell, N_neuro, N_glia, CellDen, NeuroDen, GliaDen, NeuroFrac, Neuro2Glia])

In [ ]:
MERFISH_STR_Stats = pd.DataFrame(data=dat, columns=["STR", "N_cell", "N_neuro", "N_glia", 
                                                    "CellDen", "NeuroDen", "GliaDen", 
                                                    "NeuroFrac", "Neuro2Glia"])
MERFISH_STR_Stats = MERFISH_STR_Stats.set_index("STR")

In [ ]:
MERFISH_STR_Stats

In [ ]:
for i, row in MERFISH_STR_Stats.iterrows():
    STR = "_".join(i.split())
    if STR not in ASD_MERFISH_adj.index.values:
        MERFISH_STR_Stats.loc[i, "STR2"] = ""
        MERFISH_STR_Stats.loc[i, "REGION"] = ""
    else:
        REG = ASD_MERFISH_adj.loc[STR, "REGION"]
        MERFISH_STR_Stats.loc[i, "STR2"] = STR
        MERFISH_STR_Stats.loc[i, "REGION"] = REG

In [ ]:
MERFISH_STR_Stats

In [ ]:
REGIONS_seq = ['Isocortex','Olfactory_areas', 'Cortical_subplate', 'Hippocampus','Amygdala','Striatum', "Thalamus",
              "Hypothalamus", "Midbrain", "Medulla", "Pallidum", "Pons", "Cerebellum"]

In [ ]:
sns.set(style="whitegrid")

# Create the boxplot
plt.figure(dpi=300, figsize=(12, 8))
ax = sns.boxplot(x='REGION', y='N_cell', data=MERFISH_STR_Stats, order=REGIONS_seq)

# Adding labels and title with increased font size and weight
ax.set_xlabel('Region', fontsize=14, weight='bold')
ax.set_ylabel('Number of Cells', fontsize=14, weight='bold')
#ax.set_title('Distribution of Cell Density by Region', fontsize=16, weight='bold')

# Improve the look of the grid
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Remove top and right spines
sns.despine()

# Set the tick parameters for better readability
ax.tick_params(axis='x', which='major', labelsize=20, rotation=45)
ax.tick_params(axis='y', which='major', labelsize=12)

plt.ylim(0, 100000)

# Show the plot with tight layout
plt.tight_layout()
plt.show()

In [ ]:
sns.set(style="whitegrid")

# Create the boxplot
plt.figure(dpi=300, figsize=(12, 8))
ax = sns.boxplot(x='REGION', y='CellDen', data=MERFISH_STR_Stats, order=REGIONS_seq)

# Adding labels and title with increased font size and weight
ax.set_xlabel('Region', fontsize=14, weight='bold')
ax.set_ylabel('Cell Density', fontsize=14, weight='bold')
#ax.set_title('Distribution of Cell Density by Region', fontsize=16, weight='bold')

# Improve the look of the grid
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Remove top and right spines
sns.despine()

# Set the tick parameters for better readability
ax.tick_params(axis='x', which='major', labelsize=20, rotation=45)
ax.tick_params(axis='y', which='major', labelsize=12)

# Show the plot with tight layout
plt.tight_layout()
plt.show()

In [ ]:
sns.set(style="whitegrid")

# Create the boxplot
plt.figure(dpi=300, figsize=(12, 8))
ax = sns.boxplot(x='REGION', y='NeuroDen', data=MERFISH_STR_Stats, order=REGIONS_seq)

# Adding labels and title with increased font size and weight
ax.set_xlabel('Region', fontsize=14, weight='bold')
ax.set_ylabel('Neuro Density', fontsize=14, weight='bold')
#ax.set_title('Distribution of Cell Density by Region', fontsize=16, weight='bold')

# Improve the look of the grid
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Remove top and right spines
sns.despine()

# Set the tick parameters for better readability
ax.tick_params(axis='x', which='major', labelsize=20, rotation=45)
ax.tick_params(axis='y', which='major', labelsize=12)

# Show the plot with tight layout
plt.tight_layout()
plt.show()

In [ ]:
sns.set(style="whitegrid")

# Create the boxplot
plt.figure(dpi=300, figsize=(12, 8))
ax = sns.boxplot(x='REGION', y='Neuro2Glia', data=MERFISH_STR_Stats, order=REGIONS_seq)

# Adding labels and title with increased font size and weight
ax.set_xlabel('Region', fontsize=14, weight='bold')
ax.set_ylabel('Neuro/Glia Ratio', fontsize=14, weight='bold')
#ax.set_title('Distribution of Cell Density by Region', fontsize=16, weight='bold')

# Improve the look of the grid
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Remove top and right spines
sns.despine()

# Set the tick parameters for better readability
ax.tick_params(axis='x', which='major', labelsize=20, rotation=45)
ax.tick_params(axis='y', which='major', labelsize=12)

# Show the plot with tight layout
plt.tight_layout()
plt.show()

#### STR Z1 Bias

In [ ]:
STR_MERFISH_Z1Mat_CM = pd.read_csv("dat/MERFISH/STR_Cell_Mean_Z1Mat.clip.csv", index_col=0)
STR_MERFISH_Z1Mat_VM = pd.read_csv("dat/MERFISH/STR_Vol_Mean_Z1Mat.clip.csv", index_col=0)
STR_MERFISH_QN_Z1Mat_VM = pd.read_csv("dat/MERFISH/STR_Vol_Mean_QNZ1Mat.csv", index_col=0)
STR_MERFISH_QN_Z1Mat_CM = pd.read_csv("dat/MERFISH/STR_Cell_Mean_QNZ1Mat.csv", index_col=0)
head = STR_MERFISH_Z1Mat_CM.columns.values
new_head = [X.replace(" ", "_") for X in head]
STR_MERFISH_Z1Mat_CM.columns = new_head
STR_MERFISH_Z1Mat_VM.columns = new_head
STR_MERFISH_QN_Z1Mat_VM.columns = new_head
STR_MERFISH_QN_Z1Mat_CM.columns = new_head

In [ ]:
ASD_Bias_MERFISH_STR_Z1_CM = AvgSTRZ_Weighted(STR_MERFISH_Z1Mat_CM, ASD_60_GW_adj) 
ASD_Bias_MERFISH_STR_Z1_VM = AvgSTRZ_Weighted(STR_MERFISH_Z1Mat_VM, ASD_60_GW_adj) 
ASD_Bias_MERFISH_STR_QNZ1_VM = AvgSTRZ_Weighted(STR_MERFISH_QN_Z1Mat_VM, ASD_60_GW_adj) 
ASD_Bias_MERFISH_STR_QNZ1_CM = AvgSTRZ_Weighted(STR_MERFISH_QN_Z1Mat_CM, ASD_60_GW_adj) 

In [ ]:
#ASD_Bias_MERFISH_STR_QNZ1_CM.head(60)

In [ ]:
ASD_ISH =  pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.csv", 
                               index_col=0)
ASD_ISH = FixSubiculum(ASD_ISH)

In [ ]:
Bias1 = ASD_ISH
Bias2 = ASD_Bias_MERFISH_STR_Z1_CM
#BiasCorrelation(Bias1, Bias2, "Z2", "Zmatch")
TestBiasCompare(Bias1, Bias2,  "ISH Z2", "MF Z1 CM")

#### STR Z2 Bias

In [ ]:
ASD_ISH =  pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.csv", 
                               index_col=0)
ASD_ISH = FixSubiculum(ASD_ISH)
ASD_ISH["REGION"] = ASD_ISH["REGION"].replace("Amygdalar", "Amygdala")

In [ ]:
STR_MERFISH_Z2Mat_CM = pd.read_csv("dat/MERFISH/STR_Cell_Mean_Z2Mat_ISHMatch.csv", index_col=0)
STR_MERFISH_Z2Mat_VM = pd.read_csv("dat/MERFISH/STR_Vol_Mean_Z2Mat_ISHMatch.csv", index_col=0)
STR_MERFISH_Z2Mat_NM = pd.read_csv("dat/MERFISH/STR_NEUR_Mean_Z2Mat_ISHMatch.csv", index_col=0)
STR_MERFISH_Z2Mat_NVM = pd.read_csv("dat/MERFISH/STR_NEUR_Vol_Mean_Z2Mat_ISHMatch.csv", index_col=0)
head = STR_MERFISH_Z2Mat_CM.columns.values
new_head = [X.replace(" ", "_") for X in head]
STR_MERFISH_Z2Mat_CM.columns = new_head
STR_MERFISH_Z2Mat_VM.columns = new_head
STR_MERFISH_Z2Mat_NM.columns = new_head
STR_MERFISH_Z2Mat_NVM.columns = new_head

In [ ]:
STR_MERFISH_MIT_Z2Mat_CM = pd.read_csv("dat/MERFISH_Zhuang/STR_Cell_Mean_Z2Mat_ISHMatch.csv", index_col=0)
STR_MERFISH_MIT_Z2Mat_VM = pd.read_csv("dat/MERFISH_Zhuang/STR_Vol_Mean_Z2Mat_ISHMatch.csv", index_col=0)
head = STR_MERFISH_Z2Mat_CM.columns.values
new_head = [X.replace(" ", "_") for X in head]
STR_MERFISH_MIT_Z2Mat_CM.columns = new_head
STR_MERFISH_MIT_Z2Mat_VM.columns = new_head

In [ ]:
ASD_Bias_MERFISH_MIT_STR_Z2_CM = AvgSTRZ_Weighted(STR_MERFISH_MIT_Z2Mat_CM, ASD_60_GW_adj) 
ASD_Bias_MERFISH_MIT_STR_Z2_VM = AvgSTRZ_Weighted(STR_MERFISH_MIT_Z2Mat_VM, ASD_60_GW_adj) 

##ASD_Bias_MERFISH_MIT_STR_Z2_CM = AvgSTRZ_Weighted(STR_MERFISH_MIT_Z2Mat_CM, ASD_60_GW) 
#ASD_Bias_MERFISH_MIT_STR_Z2_VM = AvgSTRZ_Weighted(STR_MERFISH_MIT_Z2Mat_VM, ASD_60_GW) 

In [ ]:
ASD_Bias_MERFISH_STR_Z2_CM = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_CM, ASD_60_GW_adj) 
ASD_Bias_MERFISH_STR_Z2_VM = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_VM, ASD_60_GW_adj) 
ASD_Bias_MERFISH_STR_Z2_NM = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_NM, ASD_60_GW_adj) 
ASD_Bias_MERFISH_STR_Z2_NVM = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_NVM, ASD_60_GW_adj) 

ASD_Bias_MERFISH_STR_Z2_CM.to_csv("dat/Bias/STR/ASD.MERFISH_Allen.CM.ISHMatch.Z2.csv")
ASD_Bias_MERFISH_STR_Z2_VM.to_csv("dat/Bias/STR/ASD.MERFISH_Allen.VM.ISHMatch.Z2.csv")
ASD_Bias_MERFISH_STR_Z2_NM.to_csv("dat/Bias/STR/ASD.MERFISH_Allen.NM.ISHMatch.Z2.csv")
ASD_Bias_MERFISH_STR_Z2_NVM.to_csv("dat/Bias/STR/ASD.MERFISH_Allen.NVM.ISHMatch.Z2.csv")
#ASD_Bias_MERFISH_STR_Z2_CM = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_CM, ASD_60_GW) 
#ASD_Bias_MERFISH_STR_Z2_VM = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_VM, ASD_60_GW) 

ASD_Bias_MERFISH_STR_Z2_CM_160 = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_CM, ASD_160_GW_adj) 
ASD_Bias_MERFISH_STR_Z2_VM_160 = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_VM, ASD_160_GW_adj) 
ASD_Bias_MERFISH_STR_Z2_NM_160 = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_NM, ASD_160_GW_adj) 
ASD_Bias_MERFISH_STR_Z2_NVM_160 = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_NVM, ASD_160_GW_adj) 

ASD_Bias_MERFISH_STR_Z2_CM_160.to_csv("dat/Bias/STR/ASD.MERFISH_Allen.CM.ISHMatch.160.Z2.csv")
ASD_Bias_MERFISH_STR_Z2_VM_160.to_csv("dat/Bias/STR/ASD.MERFISH_Allen.VM.ISHMatch.160.Z2.csv")
ASD_Bias_MERFISH_STR_Z2_NM_160.to_csv("dat/Bias/STR/ASD.MERFISH_Allen.NM.ISHMatch.160.Z2.csv")
ASD_Bias_MERFISH_STR_Z2_NVM_160.to_csv("dat/Bias/STR/ASD.MERFISH_Allen.NVM.ISHMatch.160.Z2.csv")

In [ ]:
print(RegionDistributionsList(ASD_Bias_MERFISH_STR_Z2_NM_160.head(50).index.values))

In [ ]:
print(RegionDistributionsList(ASD_Bias_MERFISH_STR_Z2_NM.head(50).index.values))

In [ ]:
print(RegionDistributionsList(ASD_Bias_MERFISH_STR_Z2_NVM.head(50).index.values))

In [ ]:
ASD_Bias_MERFISH_STR_Z2_NVM[ASD_Bias_MERFISH_STR_Z2_NVM["REGION"]=="Thalamus"]

In [ ]:
ASD_Bias_MERFISH_STR_Z2_NM[ASD_Bias_MERFISH_STR_Z2_NM["REGION"]=="Thalamus"]

In [ ]:
#ASD_Bias_MERFISH_STR_Z2_CM_60 = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_CM, ASD_60_GW) 
#ASD_Bias_MERFISH_STR_Z2_VM_60 = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_VM, ASD_60_GW) 
#ASD_Bias_MERFISH_STR_Z2_CM_160 = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_CM, ASD_160_GW) 
#ASD_Bias_MERFISH_STR_Z2_VM_160 = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_VM, ASD_160_GW) 

In [ ]:
print(RegionDistributionsList(ASD_Bias_MERFISH_STR_Z2_CM_160.head(50).index.values))

In [ ]:
print(RegionDistributionsList(ASD_Bias_MERFISH_STR_Z2_VM_160.head(50).index.values))

In [ ]:
#ASD_Bias_MERFISH_STR_Z2_CM.head(50)

In [ ]:
#print(RegionDistributionsList(ASD_Bias_MERFISH_STR_Z2_VM.head(50).index.values))

In [ ]:
#ASD_ISH[ASD_ISH["REGION"]=="Cerebellum"]
#ASD_Bias_MERFISH_STR_Z2_VM[ASD_Bias_MERFISH_STR_Z2_VM["REGION"]=="Thalamus"]

In [ ]:
Bias1 = ASD_ISH
Bias2 = ASD_Bias_MERFISH_STR_Z2_VM_160
#BiasCorrelation(Bias1, Bias2, "Z2", "Zmatch")
TestBiasCompare(Bias1, Bias2,  "ISH Z2", "MF Z2 CM")

In [ ]:
Bias1 = ASD_ISH
Bias2 = ASD_Bias_MERFISH_STR_Z2_CM
#BiasCorrelation(Bias1, Bias2, "Z2", "Zmatch")
TestBiasCompare(Bias1, Bias2,  "ISH Z2", "MF Z2 CM")

In [ ]:
Bias1 = ASD_ISH
Bias2 = ASD_Bias_MERFISH_STR_Z2_VM
#BiasCorrelation(Bias1, Bias2, "Z2", "Zmatch")
TestBiasCompare(Bias1, Bias2,  "ISH Z2", "MF Z2 VM")

In [ ]:
#ASD_Bias_MERFISH_MIT_STR_Z2_VM.head(50)

In [ ]:
Bias1 = ASD_ISH
Bias2 = ASD_Bias_MERFISH_MIT_STR_Z2_CM
#BiasCorrelation(Bias1, Bias2, "Z2", "Zmatch")
TestBiasCompare(Bias1, Bias2,  "ISH Z2", "MF MIT Z2 CM")

In [ ]:
Bias1 = ASD_ISH
Bias2 = ASD_Bias_MERFISH_MIT_STR_Z2_VM
#BiasCorrelation(Bias1, Bias2, "Z2", "Zmatch")
TestBiasCompare(Bias1, Bias2,  "ISH Z2", "MF MIT Z2 VM")

In [ ]:
Bias1 = ASD_Bias_MERFISH_STR_Z2_CM
Bias2 = ASD_Bias_MERFISH_MIT_STR_Z2_CM
#BiasCorrelation(Bias1, Bias2, "Z2", "Zmatch")
TestBiasCompare(Bias1, Bias2,  "Allen MF Z2 CM", "MIT MF Z2 CM")

In [ ]:
Bias1 = ASD_Bias_MERFISH_STR_Z2_VM
Bias2 = ASD_Bias_MERFISH_MIT_STR_Z2_VM
#BiasCorrelation(Bias1, Bias2, "Z2", "Zmatch")
TestBiasCompare(Bias1, Bias2,  "Allen MF Z2 VM", "MIT MF Z2 VM")

In [ ]:
REGIONS_seq = ['Isocortex','Olfactory_areas', 'Cortical_subplate', 'Hippocampus','Amygdala','Striatum', "Thalamus",
              "Hypothalamus", "Midbrain", "Medulla", "Pallidum", "Pons", "Cerebellum"]

In [ ]:
for Reg in REGIONS_seq:
    ISH_Var = ASD_ISH[ASD_ISH["REGION"]==Reg]["EFFECT"].var()
    MF_Var = ASD_Bias_MERFISH_STR_Z2_VM[ASD_Bias_MERFISH_STR_Z2_VM["REGION"]==Reg]["EFFECT"].var()
    print(Reg, ISH_Var, MF_Var)

In [ ]:
for STR, row in ASD_ISH.iterrows():
    ASD_ISH.loc[STR, "MF_CM_Bias"] = ASD_Bias_MERFISH_STR_Z2_CM.loc[STR, "EFFECT"]
    ASD_ISH.loc[STR, "MF_VM_Bias"] = ASD_Bias_MERFISH_STR_Z2_VM.loc[STR, "EFFECT"]

In [ ]:
print("%20s\t%10s\t%10s\t%10s" % ("Region", "rho_CM", "rho_VM", "rho_NM"))
for Reg in REGIONS_seq:
    ISH_Reg = ASD_ISH[ASD_ISH["REGION"]==Reg]
    r1, p1 = spearmanr(ISH_Reg["EFFECT"], ISH_Reg["MF_CM_Bias"])
    r2, p2 = spearmanr(ISH_Reg["EFFECT"], ISH_Reg["MF_VM_Bias"])
    
    tmp = AggV2_ASD_DF[AggV2_ASD_DF["REGION"]==Reg]
    r3, p3 = spearmanr(tmp["EFFECT_ISH"], tmp["EFFECT_perNeuro"])
    #print("%20s\t%10.2f %.2e\t%10.2f %.2e" % (Reg, r1, p1, r2, p2))
    print("%20s\t%10.2f\t%10.2f\t%10.2f" % (Reg, r1, r2, r3))

In [ ]:
REGIONS = list(set(Bias1["REGION"].values))
REG_COR_Dic = dict(zip(REGIONS, ["#B3B6B7", "#D5DBDB", "#AED6F1", "#F5B041", "#34495E", "#FAD7A0", 
                           "#884EA0", "#E6B0AA", "#33a02c",  "#E74C3C", "#2ECC71", "#D2B4DE", "#85C1E9", "#3498DB"]))

In [ ]:
ISH_REG_Bias = []
MF_REG_Bias = []
for Reg in REGIONS_seq:
    ISH_Reg = ASD_ISH[ASD_ISH["REGION"]==Reg]["EFFECT"]
    MF_Reg = ASD_Bias_MERFISH_STR_Z2_VM[ASD_Bias_MERFISH_STR_Z2_VM["REGION"]==Reg]["EFFECT"]
    #print(Reg, ISH_Reg.max() - ISH_Reg.min(), MF_Reg.max() - MF_Reg.min())
    ISH_REG_Bias.append(ISH_Reg.mean())
    MF_REG_Bias.append(MF_Reg.mean())

In [ ]:
ISH_REG_Bias = []
MF_REG_Bias = []
for Reg in REGIONS_seq:
    ISH_Reg = ASD_ISH[ASD_ISH["REGION"]==Reg]["EFFECT"]
    MF_Reg = ASD_Bias_MERFISH_STR_Z2_VM[ASD_Bias_MERFISH_STR_Z2_VM["REGION"]==Reg]["EFFECT"]
    #print(Reg, ISH_Reg.max() - ISH_Reg.min(), MF_Reg.max() - MF_Reg.min())
    ISH_REG_Bias.append(ISH_Reg.mean())
    MF_REG_Bias.append(MF_Reg.mean())
R1, P1 = pearsonr(ISH_REG_Bias, MF_REG_Bias)
R2, P2 = spearmanr(ISH_REG_Bias, MF_REG_Bias)
print(R1, P1, R2, P2)
plt.figure(dpi=200, figsize=(6,6))
for Reg in REGIONS_seq:
    ISH_Reg = ASD_ISH[ASD_ISH["REGION"]==Reg]["EFFECT"]
    MF_Reg = ASD_Bias_MERFISH_STR_Z2_VM[ASD_Bias_MERFISH_STR_Z2_VM["REGION"]==Reg]["EFFECT"]
    X = ISH_Reg.mean()
    Y = MF_Reg.mean()
    plt.scatter(ISH_Reg.mean(), MF_Reg.mean(), color=REG_COR_Dic[Reg])
    plt.text(X, Y, s=Reg, fontsize=15)
plt.plot([-0.6, 0.5], [-0.6, 0.5], color="grey", alpha=0.3)
plt.text(-0.4, 0.2, s="R = %.2f\nrho = %.2f"%(R1, R2), fontsize=15)
plt.xlabel("ISH")
plt.xlim(-0.6, 0.5)
plt.ylim(-0.6, 0.5)
plt.ylabel("MERFISH per Vol")

In [ ]:
ISH_REG_Bias = []
MF_REG_Bias = []
for Reg in REGIONS_seq:
    ISH_Reg = ASD_ISH[ASD_ISH["REGION"]==Reg]["EFFECT"]
    MF_Reg = ASD_Bias_MERFISH_STR_Z2_CM[ASD_Bias_MERFISH_STR_Z2_CM["REGION"]==Reg]["EFFECT"]
    #print(Reg, ISH_Reg.max() - ISH_Reg.min(), MF_Reg.max() - MF_Reg.min())
    ISH_REG_Bias.append(ISH_Reg.mean())
    MF_REG_Bias.append(MF_Reg.mean())
R1, P1 = pearsonr(ISH_REG_Bias, MF_REG_Bias)
R2, P2 = spearmanr(ISH_REG_Bias, MF_REG_Bias)
print(spearmanr(ISH_REG_Bias, MF_REG_Bias))
plt.figure(dpi=200, figsize=(6,6))
for Reg in REGIONS_seq:
    ISH_Reg = ASD_ISH[ASD_ISH["REGION"]==Reg]["EFFECT"]
    MF_Reg = ASD_Bias_MERFISH_STR_Z2_CM[ASD_Bias_MERFISH_STR_Z2_CM["REGION"]==Reg]["EFFECT"]
    X = ISH_Reg.mean()
    Y = MF_Reg.mean()
    plt.scatter(ISH_Reg.mean(), MF_Reg.mean(), color=REG_COR_Dic[Reg])
    plt.text(X, Y, s=Reg, fontsize=15)
plt.plot([-0.6, 0.6], [-0.6, 0.6], color="grey", alpha=0.3)
plt.text(-0.4, 0.2, s="R = %.2f\nrho = %.2f"%(R1, R2), fontsize=15)
plt.xlabel("ISH")
plt.xlim(-0.6, 0.6)
plt.ylim(-0.6, 0.6)
plt.ylabel("MERFISH Per Cell")

In [ ]:
bs_meta = pd.read_csv("../dat3/All_genes.natal.csv", index_col="entrez_id")

In [ ]:
pre_natal_Genes = bs_meta[bs_meta["peak"]=="prenatal"].index.values
post_natal_Genes = bs_meta[bs_meta["peak"]=="postnatal"].index.values

In [ ]:
ASD_GW_160 = Fil2Dict("../../ASD_Circuits/dat/Unionize_bias/Spark_Meta_160.GeneWeight.csv")
ASD_GW_160_prenatal = {}
ASD_GW_160_postnatal = {}
for g,w in ASD_GW_160.items():
    if g in pre_natal_Genes:
        ASD_GW_160_prenatal[g] = w
    elif g in post_natal_Genes:
        ASD_GW_160_postnatal[g] = w
print(len(ASD_GW_160_prenatal), len(ASD_GW_160_postnatal))

In [ ]:
ASD_160_Z2 = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_CM, ASD_GW_160_)
print(RegionDistributionsList(ASD_160_Z2.head(50).index.values))

In [ ]:
ASD_160_Z2_pre = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_CM, ASD_GW_160_prenatal)
print(RegionDistributionsList(ASD_160_Z2_pre.head(50).index.values))

In [ ]:
ASD_160_Z2_post = AvgSTRZ_Weighted(STR_MERFISH_Z2Mat_CM, ASD_GW_160_postnatal)
print(RegionDistributionsList(ASD_160_Z2_post.head(50).index.values))

In [ ]:
def TestBiasCompare(Bias1, Bias2, name1="1", name2="2"):
    
    REGIONS_seq = ['Isocortex','Olfactory_areas', 'Cortical_subplate', 'Hippocampus','Amygdala','Striatum', "Thalamus",
              "Hypothalamus", "Midbrain", "Medulla", "Pallidum", "Pons", "Cerebellum"]

    REGIONS = list(set(Bias1["REGION"].values))
    REG_COR_Dic = dict(zip(REGIONS_seq, ["#B3B6B7", "#D5DBDB", "#AED6F1", "#F5B041", "#34495E", "#FAD7A0", 
                           "#884EA0", "#E6B0AA", "#33a02c",  "#E74C3C", "#2ECC71", "#D2B4DE", "#85C1E9", "#3498DB"]))
    plt.figure(dpi=300, figsize=(6, 4))

    # Use Seaborn for style and color palette
    sns.set(style="whitegrid", context="talk")
    palette = sns.color_palette("deep")

    JoinDF = Bias1.join(Bias2, how="inner", lsuffix="_{}".format(name1), rsuffix="_{}".format(name2))
    R,P = pearsonr(JoinDF["EFFECT_{}".format(name1)].values, JoinDF["EFFECT_{}".format(name2)].values)
    RR,PP = spearmanr(JoinDF["EFFECT_{}".format(name1)].values, JoinDF["EFFECT_{}".format(name2)].values)
    # Scatter plot with improved aesthetics
    for reg in REGIONS_seq:
        RegDF = JoinDF[JoinDF["REGION_{}".format(name1)]==reg]
        X = RegDF["EFFECT_{}".format(name1)].values
        Y = RegDF["EFFECT_{}".format(name2)].values
        plt.scatter(X, Y, facecolor=REG_COR_Dic[reg], alpha=0.7, s=20, label=reg)
    minX, maxY = min(JoinDF["EFFECT_{}".format(name1)].values), max(JoinDF["EFFECT_{}".format(name2)].values)
    
    Top50_X = JoinDF.sort_values("EFFECT_{}".format(name1), ascending=False).head(50).index.values
    top50_Y = JoinDF.sort_values("EFFECT_{}".format(name2), ascending=False).head(50).index.values 
    STR_Com = len(set(Top50_X).intersection(set(top50_Y)))
    plt.text(x=minX, y=maxY*0.7, s="r=%.2f P<%.1e\nrho=%.2f P<%.1e\nSTR_Comm=%d"%(R, P, RR, PP, STR_Com), 
             fontsize=6, weight='bold')
    # Adding labels, title, and legend
    plt.xlabel(name1, fontsize=14, weight='bold')
    plt.ylabel(name2, fontsize=14, weight='bold')
    #plt.title("Scatter Plot of EFFECT vs. SC_Bias", fontsize=16, weight='bold')
    #plt.legend(fontsize=6)
    plt.legend(fontsize=8, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

    
    # Adding grid lines
    plt.grid(True, linestyle='--', alpha=0.5)

    # Adjusting ticks
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # Display the plot
    plt.tight_layout()
    plt.show()

In [ ]:
Bias1 = ASD_160_Z2_pre
Bias2 = ASD_160_Z2_post
#BiasCorrelation(Bias1, Bias2, "Z2", "Zmatch")
TestBiasCompare(Bias1, Bias2,  "MERFISH Prenatal genes", "MERFISH Postnatal genes")

In [ ]:
test = MERFISH_STRAnn[MERFISH_STRAnn["cluster"]=="0100 L4/5 IT CTX Glut_6"]

In [ ]:
test.shape

In [ ]:
test["ISH_STR"].value_counts()

In [ ]:
CT = "0101 L4/5 IT CTX Glut_6"
test = MERFISH_STRAnn[MERFISH_STRAnn["cluster"]==CT]
test["ISH_STR"].value_counts()

In [ ]:
STR = "Primary visual area"
test = MERFISH_STRAnn[MERFISH_STRAnn["ISH_STR"]==STR]
test["cluster"].value_counts().head(10)

In [ ]:
STR = "Dorsal auditory area"
test = MERFISH_STRAnn[MERFISH_STRAnn["ISH_STR"]==STR]
test["cluster"].value_counts().head(10)